互评作业二: 频繁模式与关联规则挖掘

作业说明：

使用的数据集：YELP: https://www.yelp.com/dataset/download     

代码地址：https://github.com/truth-z/DM2

In [1]:
import pandas as pd
import orangecontrib.associate.fpgrowth as oaf

一.数据集预处理

读取json数据集文件

In [2]:
# 读取json文件
# path 为数据集路径
path = 'yelp_academic_dataset_business\yelp_academic_dataset_business.json'
with open(path) as f: 
    data_raw = pd.read_json(path, lines=True)  
    f.close()
print(data_raw.columns)

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')


对读入的data进行清理，在读取的数据名称有'city'城市, 'state'州, 'stars'打分, 'review_count'评论数, 'categories'类别。

其中，对于打分和评论数这两个数值属性，使用四分之一分位数和四分之三分位数分为高、中、低三个标称数值。

In [3]:
# data = data_raw[['city', 'state', 'stars', 'review_count', 'categories']]
# print(data)

data = data_raw[['city', 'state', 'categories']]

# 对stars进行数据处理
data2 = data_raw['stars']
one = data2.quantile(0.25)
three = data2.quantile(0.75)
stars = []
for i in data2:
    if i > three:
        stars.append('high stars')
    elif i < one:
        stars.append('low stars')
    else:
        stars.append('medium stars')
# print(data2[:5])
# print(stars[:5], len(stars))

# 对review_count进行数据处理
data3 = data_raw['review_count']
one = data3.quantile(0.25)
three = data3.quantile(0.75)
review_count = []
for i in data3:
    if i > three:
        review_count.append('high reviews')
    elif i < one:
        review_count.append('low reviews')
    else:
        review_count.append('medium reviews')
# print(data3[:5])
# print(review_count[:5], len(review_count))

data.insert(0, 'stars', stars)
data.insert(0, 'review_count', review_count)
N = len(data)
print(data.columns, len(data))

Index(['review_count', 'stars', 'city', 'state', 'categories'], dtype='object') 150346


二.关联关系挖掘

In [4]:
# 输入格式转换
int_to_str = {}     # 整数编码 —> 字符串
str_to_int= {}      # 字符串 -> 整数编码
idx = 0
data_int = []
cols = data.columns
for i in range(len(data)):
    tmp = []
    for col in cols:
        if col == 'categories' and data.loc[i, col] != None:
            for j in data.loc[i, col].split(', '):
                if j in str_to_int:
                    tmp.append(str_to_int[j])
                else:
                    int_to_str[idx] = j
                    str_to_int[j] = idx
                    tmp.append(idx)
                    idx += 1
        elif data.loc[i, col] in str_to_int:
            tmp.append(str_to_int[data.loc[i, col]])
        else:
            int_to_str[idx] = data.loc[i, col]
            str_to_int[data.loc[i, col]] = idx
            tmp.append(idx)
            idx += 1
    # print(tmp)
    # print()
    data_int.append(tmp)
print(data[:5])
print(data_int[:5])

     review_count         stars           city state  \
0     low reviews    high stars  Santa Barbara    CA   
1  medium reviews  medium stars         Affton    MO   
2  medium reviews  medium stars         Tucson    AZ   
3    high reviews  medium stars   Philadelphia    PA   
4  medium reviews  medium stars     Green Lane    PA   

                                          categories  
0  Doctors, Traditional Chinese Medicine, Naturop...  
1  Shipping Centers, Local Services, Notaries, Ma...  
2  Department Stores, Shopping, Fashion, Home & G...  
3  Restaurants, Food, Bubble Tea, Coffee & Tea, B...  
4                          Brewpubs, Breweries, Food  
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [10, 11, 12, 13, 14, 15, 16, 17, 18], [10, 11, 19, 20, 21, 22, 23, 24, 25, 26], [27, 11, 28, 29, 30, 31, 32, 33, 34], [10, 11, 35, 29, 36, 37, 31]]


频繁项集挖掘

In [5]:
result_supp = dict(oaf.frequent_itemsets(data_int, .08)) #支持度
print(result_supp)
print(len(result_supp))


{frozenset({0}): 36188, frozenset({1}): 16307, frozenset({10}): 76989, frozenset({11}): 103278, frozenset({0, 11}): 20419, frozenset({10, 11}): 51567, frozenset({22}): 24395, frozenset({10, 22}): 13967, frozenset({11, 22}): 16219, frozenset({27}): 37169, frozenset({27, 11}): 31292, frozenset({28}): 14569, frozenset({29}): 34039, frozenset({10, 29}): 17195, frozenset({11, 29}): 23432, frozenset({28, 29}): 14567, frozenset({30}): 52268, frozenset({10, 30}): 22224, frozenset({11, 30}): 41228, frozenset({10, 11, 30}): 15688, frozenset({27, 30}): 24440, frozenset({27, 11, 30}): 21837, frozenset({29, 30}): 12641, frozenset({31}): 27781, frozenset({10, 31}): 13542, frozenset({11, 31}): 21132, frozenset({30, 31}): 15472, frozenset({11, 30, 31}): 12308, frozenset({38}): 30761, frozenset({10, 38}): 17554, frozenset({40}): 12056, frozenset({51}): 26330, frozenset({10, 51}): 13489, frozenset({51, 11}): 18058, frozenset({58}): 12281, frozenset({153}): 14292, frozenset({188}): 14356}
37


In [6]:
# [频繁项，支持度]
items = []
for i in result_supp:
    # print(i)
    tmp = []
    for j in i:
        tmp.append(int_to_str[j])
    items.append([tmp, round(result_supp[i]/N, 4)])
    # print(items[-1])

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df = pd.DataFrame(items, columns=['频繁项集', '支持度']) 
df = df.sort_values('支持度',ascending=False)
df.index = range(len(df))
print(df)

                                           频繁项集     支持度
0                                [medium stars]  0.6869
1                              [medium reviews]  0.5121
2                                 [Restaurants]  0.3477
3                [medium reviews, medium stars]  0.3430
4                   [medium stars, Restaurants]  0.2742
5                                [high reviews]  0.2472
6                                 [low reviews]  0.2407
7                                          [PA]  0.2264
8                  [high reviews, medium stars]  0.2081
9                                   [low stars]  0.2046
10                                       [Food]  0.1848
11                                         [FL]  0.1751
12                  [high reviews, Restaurants]  0.1626
13                                   [Shopping]  0.1623
14                           [medium stars, PA]  0.1559
15                [medium reviews, Restaurants]  0.1478
16    [high reviews, medium stars, Restaurants] 

关联规则挖掘

In [7]:
result_rules = oaf.association_rules(result_supp, .5)   #置信度
result_rules = list(result_rules)
print(len(result_rules))
result_rules

22


[(frozenset({10, 30}), frozenset({11}), 15688, 0.7059035277177825),
 (frozenset({11, 30}), frozenset({27}), 21837, 0.5296643058115844),
 (frozenset({27, 30}), frozenset({11}), 21837, 0.893494271685761),
 (frozenset({11, 27}), frozenset({30}), 21837, 0.6978460948485236),
 (frozenset({27}), frozenset({11, 30}), 21837, 0.5875057171298663),
 (frozenset({30, 31}), frozenset({11}), 12308, 0.795501551189245),
 (frozenset({11, 31}), frozenset({30}), 12308, 0.5824342229793678),
 (frozenset({0}), frozenset({11}), 20419, 0.564247816955897),
 (frozenset({10}), frozenset({11}), 51567, 0.6697969839847251),
 (frozenset({22}), frozenset({10}), 13967, 0.5725353556056569),
 (frozenset({22}), frozenset({11}), 16219, 0.6648493543758967),
 (frozenset({27}), frozenset({11}), 31292, 0.841884366003928),
 (frozenset({29}), frozenset({10}), 17195, 0.5051558506419107),
 (frozenset({29}), frozenset({11}), 23432, 0.6883868503775081),
 (frozenset({28}), frozenset({29}), 14567, 0.999862722218409),
 (frozenset({30}),

In [8]:
# [规则前项，规则后项，支持度，置信度]
Rules = []
for i in result_rules:
    tmp1 = []
    tmp2 = []
    for j in i[0]:   #处理第一个frozenset
        tmp1.append(int_to_str[j])
    for j in i[1]:
        tmp2.append(int_to_str[j])
    Rules.append([tmp1, tmp2, round(i[2]/N, 4), round(i[3], 4)])
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df = pd.DataFrame(Rules, columns=['关联规则前项', '关联规则后项', '支持度', '置信度']) 
df = df.sort_values('置信度',ascending=False)
df.index = range(len(df))
print(df)

                           关联规则前项                       关联规则后项     支持度     置信度
0                  [Philadelphia]                         [PA]  0.0969  0.9999
1     [high reviews, Restaurants]               [medium stars]  0.1452  0.8935
2                  [high reviews]               [medium stars]  0.2081  0.8419
3             [Restaurants, Food]               [medium stars]  0.0819  0.7955
4                   [Restaurants]               [medium stars]  0.2742  0.7888
5                          [Food]               [medium stars]  0.1406  0.7607
6   [medium reviews, Restaurants]               [medium stars]  0.1043  0.7059
7    [medium stars, high reviews]                [Restaurants]  0.1452  0.6978
8                            [PA]               [medium stars]  0.1559  0.6884
9                            [FL]               [medium stars]  0.1201  0.6858
10               [medium reviews]               [medium stars]  0.3430  0.6698
11                     [Shopping]               [med

In [9]:
results = list(oaf.rules_stats(result_rules, result_supp, len(data_int)))

resultsRules = []
for i in results:
    tmp1 = []
    tmp2 = []
    for j in i[0]:  
        tmp1.append(int_to_str[j])
    for j in i[1]:
        tmp2.append(int_to_str[j])
    resultsRules.append([tmp1, tmp2, round(i[2]/N, 4), round(i[3], 4), round(i[6], 4), round(i[7], 4)])
df = pd.DataFrame(resultsRules, columns=['关联规则前项', '关联规则后项', '支持度', '置信度', 'lift', 'leverage']) 
df = df.sort_values('lift',ascending=False)
df.index = range(len(df))
df

,关联规则前项,关联规则后项,支持度,置信度,lift,leverage
0,[Philadelphia],[PA],0.0969,0.9999,4.4163,0.0750
1,[high reviews],"[medium stars, Restaurants]",0.1452,0.5875,2.1425,0.0775
2,"[medium stars, Restaurants]",[high reviews],0.1452,0.5297,2.1425,0.0775
3,"[medium stars, high reviews]",[Restaurants],0.1452,0.6978,2.0073,0.0729
4,[high reviews],[Restaurants],0.1626,0.6575,1.8914,0.0766
5,"[medium stars, Food]",[Restaurants],0.0819,0.5824,1.6753,0.0330
6,[Food],[Restaurants],0.1029,0.5569,1.6020,0.0387
7,"[high reviews, Restaurants]",[medium stars],0.1452,0.8935,1.3007,0.0336
8,[high reviews],[medium stars],0.2081,0.8419,1.2256,0.0383
9,"[Restaurants, Food]",[medium stars],0.0819,0.7955,1.1580,0.0112


三.模式命名

商家的评论和评分模式：挖掘商家的评分、评论与商家所在的州、城市、类别之间的联系



四.可视化




In [10]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df = pd.DataFrame(items, columns=['频繁项集', '支持度']) 
df = df.sort_values('支持度',ascending=False)
df.index = range(len(df))
print('频繁项集：')
print(df)

频繁项集：
                                           频繁项集     支持度
0                                [medium stars]  0.6869
1                              [medium reviews]  0.5121
2                                 [Restaurants]  0.3477
3                [medium reviews, medium stars]  0.3430
4                   [medium stars, Restaurants]  0.2742
5                                [high reviews]  0.2472
6                                 [low reviews]  0.2407
7                                          [PA]  0.2264
8                  [high reviews, medium stars]  0.2081
9                                   [low stars]  0.2046
10                                       [Food]  0.1848
11                                         [FL]  0.1751
12                  [high reviews, Restaurants]  0.1626
13                                   [Shopping]  0.1623
14                           [medium stars, PA]  0.1559
15                [medium reviews, Restaurants]  0.1478
16    [high reviews, medium stars, Restaur

In [11]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df = pd.DataFrame(Rules, columns=['关联规则前项', '关联规则后项', '支持度', '置信度']) 
df = df.sort_values('置信度',ascending=False)
df.index = range(len(df))
print("关联规则")
print(df)

关联规则
                           关联规则前项                       关联规则后项     支持度     置信度
0                  [Philadelphia]                         [PA]  0.0969  0.9999
1     [high reviews, Restaurants]               [medium stars]  0.1452  0.8935
2                  [high reviews]               [medium stars]  0.2081  0.8419
3             [Restaurants, Food]               [medium stars]  0.0819  0.7955
4                   [Restaurants]               [medium stars]  0.2742  0.7888
5                          [Food]               [medium stars]  0.1406  0.7607
6   [medium reviews, Restaurants]               [medium stars]  0.1043  0.7059
7    [medium stars, high reviews]                [Restaurants]  0.1452  0.6978
8                            [PA]               [medium stars]  0.1559  0.6884
9                            [FL]               [medium stars]  0.1201  0.6858
10               [medium reviews]               [medium stars]  0.3430  0.6698
11                     [Shopping]              




五.挖掘结果分析

根据频繁项集可以看出0.6869的商家有着中等量的评论，0.5121的商家有着相对中等的评分，这和medium设计的初衷相符。此外，数据集中类别中数量最多的餐厅，占比0.3477，且中等评分的餐厅占比0.2742。

根据挖掘的关联规则可以看出，Philadelphia城市大概率位于PA州。同时，高评论数量的数据中，有0.5875的占比是中等评分的餐厅，且在中等评分的餐厅中有0.5297的占比是高评论。